# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [ ]:
import praw
from praw.models import MoreComments
import pandas as pd
reddit = praw.Reddit(client_id = "lfdF-I0nP51611Fidk3OQw", client_secret= "MShIIjVvaY-WZIdvvB-qTCWZ9dPhZw", user_agent= "epds_project")

We can now access the subreddit, extract all the posts url in it and check which other subreddit reposted that posts and how much:

In [ ]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

for i in reddit.subreddit("conspiracy").top(limit=5000): #recupera i 5000 post top di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + il permalink


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        subreddit_url = str(repost.subreddit)
        subreddit_url = "https://www.reddit.com/r/" + subreddit_url
        if subreddit_url in conspiracy_dict.keys():
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            conspiracy_dict[subreddit_url][1][0] +=1
        else:
            conspiracy_dict[subreddit_url]=[[],[1]]
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_data/conspiracy_top_url.csv',index=False)




# Creating the network's first level:
Reading the csv we can now look for posts that have the same url as the ones in the file: this is done in order to later create a network where the subreddits will serve as nodes and the crossposts as edges between them (the more the crossposts the wider the edge).
First we need to look at all the files in the directory.

In [ ]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

Now we can scrape stuff from reddit.

In [ ]:
df1 = pd.read_csv('results/conspiracy_data/conspiracy_top_url.csv')
id_to_analyse = []
for column in df1.columns:
    value = df1[column][1][1:-1]
    if int(value) >= 5:
        ind = column.index('/r/')
        id = column[ind+3:]
        id_to_analyse.append(id)
        

for subreddit in id_to_analyse:
    post_list=list()
    subreddit_list = list()
    conspiracy_dict=dict() 
    for i in reddit.subreddit(subreddit).top(limit=5000):
        post_list.append((i.title, i.score, i.url))



    for post in post_list:
        for repost in reddit.subreddit('all').search('url:'+post[2]):
            if repost.subreddit_id != "t5_"+reddit.subreddit(subreddit).id: #cosa facciamo?
                subreddit_url = str(repost.subreddit)
                subreddit_url = "https://www.reddit.com/r/" + subreddit_url
                if subreddit_url in conspiracy_dict.keys():
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                    conspiracy_dict[subreddit_url][1][0] +=1
                else:
                    conspiracy_dict[subreddit_url]=[[],[1]]
                    conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
    df = pd.DataFrame(conspiracy_dict)
    df.to_csv(r'results/1st_level/'+subreddit+'.csv')

# Creating the second level
Now we need to iterate again on the subreddit we found in order to get where this path, started from conspiracy will led.\
We also need to create a function to dump done csv.

In [3]:
import pandas as pd
top_comments_list = dict()
to_scan = [file for file in get_all_in_dir("results/1st_level_2")]
to_avoid= [file.split('\\')[1] for file in get_all_in_dir("results/1st_level")]
to_avoid.extend(file.split('\\')[1] for file in get_all_in_dir("results/1st_level_2"))
to_avoid.extend(file.split('\\')[1] for file in get_all_in_dir("results/2nd_level"))

for subr in to_scan:
        id_to_analyse = []
        print(f'now opening {subr}')
        try:
                df1 = pd.read_csv(subr)
        except:
                pass
        try:
                df1 = pd.read_csv(subr, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        for column in df1.columns:
                try: 
                        value = df1[column][1][1:-1]
                except:
                        continue
                if int(value) >= 5:
                        ind = column.index('/r/')
                        id = column[ind+3:]
                        if not id + '.csv' in to_avoid:
                                try:
                                        post_list=list()
                                        subreddit_list = list()
                                        conspiracy_dict=dict() 
                                        for i in reddit.subreddit(id).top(limit=500):
                                                post_list.append((i.title, i.score, i.url))

                                        for post in post_list:
                                                for repost in reddit.subreddit('all').search('url:'+post[2]):
                                                        if repost.subreddit_id != "t5_"+reddit.subreddit(id).id: #cosa facciamo?
                                                                subreddit_url = str(repost.subreddit)
                                                                subreddit_url = "https://www.reddit.com/r/" + subreddit_url
                                                        else:
                                                                continue
                                                        if subreddit_url in conspiracy_dict.keys():
                                                                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                                                                conspiracy_dict[subreddit_url][1][0] +=1
                                                        else:
                                                                conspiracy_dict[subreddit_url]=[[],[1]]
                                                                conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
                                        
                                        df = pd.DataFrame(conspiracy_dict)
                                        df.to_csv(r'results/2nd_level/'+id+'.csv')
                                        with open("results/2nd_level/done_2.txt",'a', encoding = "utf-8") as text_note:
                                                text_note.write(id + "\n")
                                                text_note.close()
                                        
                                except Exception as E:
                                        print(E)
                                        with open("results/2nd_level/error_2.txt",'a', encoding = "utf-8") as text_note:
                                                text_note.write(id + "\n")
                                                text_note.close()
                                

KeyboardInterrupt: 

# Cleaning the data
In order to perform more efficiently the operations of data representation we decided two perform two operations:
- 1st we removed all columns belonging to subreddits with an amount of crossposts inferior to 5
- 2nd we decided to remove the  [ ] for the number of crossopst between channels

These was in order to have cleaner and more usable data

In [ ]:
#removing parentheses and connections with less than 5 reposts

datasets= [file for file in get_all_in_dir("results/1st_level")]
datasets.extend(file for file in get_all_in_dir("results/1st_level_2"))
datasets.extend(file for file in get_all_in_dir("results/2nd_level"))
datasets.extend("results/conspiracy_data/conspiracy_top_url.csv")

for file in datasets:
    df = pd.read_csv(file)
    for col in df.columns:
        if "u/" in col:
            df.drop(col, inplace=True, axis=1)
        elif isinstance(df[col][1], str):
            df[col][1]=int(df[col][1][1:-1])
            if df[col][1]<5:
                df.drop(col, inplace=True, axis=1)
        else:
            df.drop(col, inplace=True, axis=1)
    #df.to_csv(file, encoding="utf8")
        

# DOBBIAMO ARRIVARE AD AVERE UN PERMALINK ASSOCIATO ALLA SENTIMENT DEI COMMENTI E DEL POST


# Topic Modelling
To extract topics discussed directly from posts in reddit we decided to rely on Gensim, a package runnable in python that is comparable to Mallet in accuracy.

In [ ]:
import nltk
#nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['re', 'edit']) 

import re
import os
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as genmodels  # don't skip this
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Create the dataset

In [ ]:
import random
def iterSample(iterable, samplesize):
    results = []
    for i, v in enumerate(iterable):
        r = random.randint(0, i)
        if r < samplesize:
            if i < samplesize:
                results.insert(r, v) # add first samplesize items in random order
            else:
                results[r] = v # at a decreasing rate, replace random items

    if len(results) < samplesize:
        raise ValueError("Sample larger than population.")

    return results

Now we need to prepare the stopwords and import the datsets

In [ ]:
# Import Dataset
dir = "results/1st_level"
top_comments_list = dict()
to_scan = [] #iterate over this
for filename in os.listdir(dir):
    f = os.path.join(dir, filename)
    if os.path.isfile(f) and f[-4:] == ".csv":
        to_scan.append(f) 
for subreddit in to_scan:
        try:
                df = pd.read_csv(subreddit, sep=';', encoding='unicode_escape')
        except:
                pass
        try:
                df = pd.read_csv(subreddit, sep=',', encoding='utf8')
        except:
                pass
        try:
                df = pd.read_csv(subreddit, sep=';', encoding='utf8')
        except:
                pass
        try:
                df = pd.read_csv(subreddit, sep=',', encoding='unicode_escape')
        except:
                print(f'{subreddit} should be opened with different settings.')
                continue
        for column in df.columns: # qui inserire un loop sui permalink
                list_post=[]
                cross_counter = column[1]
                try:
                        list_post= df[column][0].split(',')
                except:
                        try:
                                continue
                        except:
                                print(f'{subreddit} has problems in formatting.')
                                pass
                for post_url in list_post:
                        tmp = post_url 
                        tmp =  re.sub("'[]", '', tmp)
                        
                        try:
                                post = reddit.submission(url=tmp)
                        except: # some posts may be removed
                                continue
                        try:
                                post.comments.replace_more(limit=0)
                        except:
                                print(f'{tmp} was not accessible')
                                continue
                        result = []
                        if len(post.comments)> 500:
                                for comment in iterSample(post.comments, 500):
                                        result.append(comment)
                        else:
                                for comment in post.comments:
                                        result.append(comment)

                        name= subreddit + " @ " + column + " <" + cross_counter + ">"            

                        if name in top_comments_list.keys():
                                top_comments_list[name].extend(result)
                        else:
                                top_comments_list[name]=result

We can now perform the topic modeling: first we need to define the functions.

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


We can now print a csv for each crossposting between subreddits with the topic modeling of the common posts.

In [ ]:

for key in top_comments_list.keys():

    # Convert to list
    data = top_comments_list[key]

    # Remove tags from comments
    data = [re.sub('\S*@\S*\s?', '', sent.body) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    # Tokenizing the text
    data_words = list(sent_to_words(data))

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # Human readable format of corpus (term-frequency)
    [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

    if corpus != [] and corpus != [[]]:
        # Build LDA model
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=10, 
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)

        # Print the Keyword in the 10 topics
        df_topic = pd.DataFrame(lda_model.print_topics())
        df_topic.to_csv(r"results/topic_models/1st_level"+ key)

        #pprint(lda_model.print_topics())

        doc_lda = lda_model[corpus]
    else: 
        print (key + " is done.")